In [1]:
# Model deployment in tensorflow and keras

#
#We will learn how to deploy a machine learning model to production with tensorflow serving
#
# Steps to load a tensorflow model into prodution includes the following



# A. The TensorFlow model needs to be trained by activating the tensorflow graph under an active session.
# B. Once the model is trained, TensorFlow’s SavedModelBuilder module can be used to export the model
    # NB: FLAGS.model_version is used to specify the model version
# C. The model, once exported, can be loaded using Standard TensorFlow Model Server,or one can choose to use a locally compiled model server.

In [1]:
# Illustration of How to Save a Model in TensorFlow

# making all necessary imports

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

batch_size,learning_rate = 256,0.001
epochs = 10
total_iter = 1000

# defining variables and placeholders
x = tf.placeholder(tf.float32,[None,784],name='x')
y = tf.placeholder(tf.float32,[None,10],name='y')
W = tf.Variable(tf.random_normal([784,10],mean=0,stddev=0.02),name='W')
b = tf.Variable(tf.random_normal([10],mean=0,stddev=0.02),name='b')


logits = tf.add(tf.matmul(x,W),b,name='logits')
# classifiation layer using softmax activation function

#NB: this is a multi-class classification task

pred = tf.nn.softmax(logits,name='pred')

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(pred,1),name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name='accuracy')


mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

batches = (mnist.train.num_examples//batch_size)

# Very important line below: Instantiating the model Saver
# will automatically save the whole session metagraph along 
# with the values for the variables in the location specified

saver = tf.train.Saver()
##############################
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=y))
optimizer_ = tf.train.AdamOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for step in range(total_iter):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer_,feed_dict={x:batch_x,y:batch_y})
        
        c = sess.run(cost,feed_dict={x:batch_x,y:batch_y})
        
        print ('Loss in iteration ' + str(step) + '= ' + str(c))
        
        if step % 100 == 0 :
            saver.save(sess,'./model_base',global_step=step)
    saver.save(sess,'/home/santanu/model_base',global_step=step)
    
    val_x,val_y = mnist.test.next_batch(batch_size)
    
    print('Accuracy:',sess.run(accuracy,feed_dict={x:val_x,y:val_y}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.


URLError: <urlopen error [Errno 110] Connection timed out>

In [ ]:
# Illustration of Restoring a Saved Model in TensorFlow

batch_size = 256
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    new_saver = tf.train.import_meta_graph('/home/santanu/model_basic-999.meta')
    new_saver.restore(sess,tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph()
    pred = graph.get_tensor_by_name("pred:0")
    accuracy = graph.get_tensor_by_name("accuracy:0")
    x = graph.get_tensor_by_name("x:0")
    y = graph.get_tensor_by_name("y:0")
    val_x,val_y = mnist.test.next_batch(batch_size)
    pred_out = sess.run(pred,feed_dict={x:val_x})
    accuracy_out = sess.run(accuracy,feed_dict={x:val_x,y:val_y})
    print 'Accuracy on Test dataset:',accuracy_out